In [69]:
import polars as pl
import os
import re

In [70]:
!pwd

/teamspace/studios/this_studio


In [71]:
os.makedirs("seismic_data/data/test/mars", exist_ok=True)
os.makedirs("seismic_data/data/test/moon", exist_ok=True)
os.makedirs("seismic_data/data/train/mars", exist_ok=True)
os.makedirs("seismic_data/data/train/moon", exist_ok=True)

In [72]:
catalog: pl.DataFrame = pl.read_csv(
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv"
)
catalog

filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
str,str,f64,str,str
"""xa.s12.00.mhz.1970-01-19HR00_e…","""1970-01-19T20:25:00.000000""",73500.0,"""evid00002""","""impact_mq"""
"""xa.s12.00.mhz.1970-03-25HR00_e…","""1970-03-25T03:32:00.000000""",12720.0,"""evid00003""","""impact_mq"""
"""xa.s12.00.mhz.1970-03-26HR00_e…","""1970-03-26T20:17:00.000000""",73020.0,"""evid00004""","""impact_mq"""
"""xa.s12.00.mhz.1970-04-25HR00_e…","""1970-04-25T01:14:00.000000""",4440.0,"""evid00006""","""impact_mq"""
"""xa.s12.00.mhz.1970-04-26HR00_e…","""1970-04-26T14:29:00.000000""",52140.0,"""evid00007""","""deep_mq"""
…,…,…,…,…
"""xa.s12.00.mhz.1974-10-14HR00_e…","""1974-10-14T17:43:00.000000""",63780.0,"""evid00156""","""impact_mq"""
"""xa.s12.00.mhz.1975-04-12HR00_e…","""1975-04-12T18:15:00.000000""",65700.0,"""evid00191""","""impact_mq"""
"""xa.s12.00.mhz.1975-05-04HR00_e…","""1975-05-04T10:05:00.000000""",36300.0,"""evid00192""","""impact_mq"""


In [73]:
catalog["time_rel(sec)"][0]

73500.0

# Lunar


## Training


In [74]:
training_dir = (
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/training/data/S12_GradeA"
)

In [75]:
for i in range(len(catalog)):
    try:
        eventID = re.search("evid(\d+)", catalog["filename"][i]).group(1)
        data = pl.read_csv(f"{training_dir}/{catalog['filename'][i]}.csv")
        velocity_data = data["velocity(m/s)"]
        time_rel = data["time_rel(sec)"]
        mq_time_rel = catalog["time_rel(sec)"][i]
        MQ_TIME = data.with_columns(
            pl.when(pl.col("time_rel(sec)") >= mq_time_rel).then(1).otherwise(0)
        )["literal"]

        dictionary = {
            "mq": MQ_TIME.to_list(),
            "time_rel": time_rel.to_list(),
            "velocity": velocity_data.to_list(),
        }
        df = pl.DataFrame(dictionary)
        df.write_csv(f"seismic_data/data/train/moon/12_a_{eventID}.csv")
    except Exception as e:
        print(catalog["filename"][i] + " failed " + str(object=e))

xa.s12.00.mhz.1971-04-13HR00_evid00029 failed No such file or directory (os error 2): ...tion_data/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv


## Testing


In [76]:
testing_dir = "nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/"
testing_dirs = [testing_dir + d for d in os.listdir(testing_dir)]

In [77]:
testing_dirs

['nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S12_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeB']

In [78]:
for j in range(len(testing_dirs)):
    for i in range(len(os.listdir(testing_dirs[j]))):
        try:
            filename = f"{testing_dirs[j]}/{os.listdir(testing_dirs[j])[i]}"
            s = re.search("s(\d+)", filename).group(1)
            grade = re.search("Grade([A-Z])", filename).group(1).lower()
            eventID = re.search("evid(\d+)", filename).group(1)
            if filename.endswith(".csv") is not True:
                continue
            data = pl.read_csv(filename)
            velocity_data = data["velocity(m/s)"]
            time_rel = data["time_rel(sec)"]
            dictionary = {
                "mq": [0] * len(time_rel),
                "time_rel": time_rel.to_list(),
                "velocity": velocity_data.to_list(),
            }
            df = pl.DataFrame(dictionary)
            df.write_csv(f"seismic_data/data/test/moon/{s}_{grade}_{eventID}.csv")
        except Exception as e:
            print(filename + " failed " + str(object=e))

# mars


## Training


In [79]:
training_dir = "nasa_space_apps_2024_seismic_detection_data/data/mars/training/data"
catalog = pl.read_csv(
    "nasa_space_apps_2024_seismic_detection_data/data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv"
)
catalog

filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid
str,str,f64,str
"""XB.ELYSE.02.BHV.2022-02-03HR08…","""2022-02-03T08:08:27.000000""",507.0,"""evid0005"""
"""XB.ELYSE.02.BHV.2022-01-02HR04…","""2022-01-02T04:35:30.000000""",2130.0,"""evid0006"""


In [80]:
for i in range(len(catalog)):
    try:
        eventID = re.search("evid(\d+)", catalog["filename"][i]).group(1)

        data = pl.read_csv(f"{training_dir}/{catalog['filename'][i]}")
        velocity_data = data["velocity(c/s)"]
        time_rel = data["rel_time(sec)"]
        mq_time_rel = catalog["time_rel(sec)"][i]
        MQ_TIME = data.with_columns(
            pl.when(pl.col("rel_time(sec)") >= mq_time_rel).then(1).otherwise(0)
        )["literal"]

        dictionary = {
            "mq": MQ_TIME.to_list(),
            "time_rel": time_rel,
            "velocity": velocity_data.to_list(),
        }
        df = pl.DataFrame(dictionary)
        df.write_csv(f"seismic_data/data/train/mars/{eventID}.csv")
    except Exception as e:
        print(catalog["filename"][i] + " failed " + str(object=e))

## Testing


In [81]:
testing_dirs = ["nasa_space_apps_2024_seismic_detection_data/data/mars/test/data"]

In [82]:
testing_dirs

['nasa_space_apps_2024_seismic_detection_data/data/mars/test/data']

In [83]:
for j in range(len(testing_dirs)):
    for i in range(len(os.listdir(testing_dirs[j]))):
        try:
            filename = f"{testing_dirs[j]}/{os.listdir(testing_dirs[j])[i]}"
            eventID = re.search("evid(\d+)", filename).group(1)
            if filename.endswith(".csv") is not True:
                continue
            # print(filename)
            # print(eventID)

            data = pl.read_csv(filename)
            velocity_data = data["velocity(c/s)"]
            time_rel = data["rel_time(sec)"]
            dictionary = {
                "mq": [0] * len(time_rel),
                "time_rel": time_rel.to_list(),
                "velocity": velocity_data.to_list(),
            }
            
            df = pl.DataFrame(dictionary)
            df.write_csv(f"seismic_data/data/test/mars/{eventID}.csv")
        except Exception as e:
            print(filename + " failed " + str(object=e))